In [ ]:
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [ ]:
df = pd.read_csv('../data/life_expectancy_data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
# Cleaning columns name to avoid manipulation errors (remove useless spaces, lowercase, replace interspace by dash)

df.columns = [i.strip().lower().replace(' ','_') for i in df.columns]
df = df.rename(columns={'thinness__1-19_years':'thinness_1-19_years'})

## Adding a linear trend

In [ ]:
result = zip(sorted(df.year.unique()),range(1,17))
dic_year = dict(result)

In [ ]:
df['year_trend'] = df.year.replace(dic_year)

In [ ]:
df.head()

## Correction of country status

In [ ]:
df[df.status=='Developing']['country'].unique()

In [ ]:
# Get a list of developed countries from Wikipedia comparative table to correct the information

text="""Lithuania
Latvia
 Estonia
 Israel
 Slovenia
 Czech Republic
 Slovak Republic
 South Korea
 Portugal
 Greece
 New Zealand
 Spain
 Ireland
 Finland
 Iceland
 United Kingdom
 Sweden
 Italy
 Australia
 Canada
 Belgium
 France
 Japan
 Luxembourg
 Austria
 Netherlands
 Germany
 Denmark
 United States
 Norway
Switzerland"""

developed_countries = text.strip().split()

In [ ]:
# Get a list of under-developed countries from Wikipedia comparative table to correct the information

text2="""Angola
Benin
Burkina Faso
Burundi
Central African Republic
Chad
Comoros
Democratic Republic of the Congo
Djibouti
Eritrea
Ethiopia
Gambia
Guinea
Guinea-Bissau
Lesotho
Liberia
Madagascar
Malawi
Mali
Mauritania
Mozambique
Niger
Rwanda
Sao Tome and Principe
Senegal
Sierra Leone
Somalia
South Sudan
Sudan
Tanzania
Togo
Uganda
Zambia
Haiti
Afghanistan
Bangladesh
Bhutan
Cambodia
East Timor
Laos
Myanmar
Nepal
Yemen
Kiribati
Solomon Islands
Tuvalu
Vanuatu"""

least_developed = text2.strip().split()

In [ ]:
# Correct the status of these countries that are false

def fix_status(country):
    
    global developed_countries
    global Least_developed
    
    if country in developed_countries:
        return 'Developed'
    if country in least_developed:
        return 'Least_developed'
    else:
        return 'Developing'
    
# Testing the function
fix_status('Canada')


In [ ]:
df.status = df.apply(lambda x: fix_status(x['country']) if x['status']=='Developing' else x['status'], axis=1)

In [ ]:
df.status.value_counts()

In [ ]:
df[df.status=='Developed']['country'].unique()

## Checking missing values

In [ ]:
df.isna().sum()

In [ ]:
# checking the country having more than 6 nan values in the row (so missing values for 6 columns)

"""
Here I think we should drop these rows because it will
On 22 columns, we have 17 relevant columns so 6 is about 35% of missing values for the given country
"""

rows_to_drop = df[df['country'].isin(df.loc[df.isna().sum(axis=1)>6]['country'].unique())].index
df.drop(index=rows_to_drop,inplace=True)

In [ ]:
df.shape

In [ ]:
# Checking null values for schooling

"""
No value available from the country when missing values. 
What should we do? Drop the rows for the given country or replacing the values by mode of the same continent?
"""

print(len(df[df.schooling.isna()]['country'].unique()))
df[df.schooling.isna()]['country'].unique()

In [ ]:
# checking null values for the income composition of resources

print(len(df[df['income_composition_of_resources'].isna()]['country'].unique()))
df[df['income_composition_of_resources'].isna()]['country'].unique()

In [ ]:
# checking null values for thinness 5-9 years and 1-19 years

print(len(df[df['thinness_5-9_years'].isna()]['country'].unique()))
print(df[df['thinness_5-9_years'].isna()]['country'].unique())
print(len(df[df['thinness_1-19_years'].isna()]['country'].unique()))
print(df[df['thinness_1-19_years'].isna()]['country'].unique())

In [ ]:
# Adding these 2 columns because income composition of resources makes no sense and 
# thinness 5-9 years is already included in thinness 1-19 years

columns_to_drop=[] # Creating a list to store the columns to drop
columns_to_drop.extend(['income_composition_of_resources','thinness_5-9_years'])

In [ ]:
# checking null values for population

"""
Here we may drop the column because it has no sense without the size of the country
"""

print(len(df[df['population'].isna()]['country'].unique()))
df[df['population'].isna()]['country'].unique()

In [ ]:
columns_to_drop.append('population')

In [ ]:
# checking null values for Gross Domestic Product per capita (in USD)

"""
A lot of missing values. We should drop this value because it makes no sense as it is dependant of the population.
"""

print(len(df[df['gdp'].isna()]['country'].unique()))
df[df['gdp'].isna()]['country'].unique()

In [ ]:
columns_to_drop.append('gdp')

In [ ]:
# checking null values for the income composition of resources

"""
Should we drop these rows because we wouldn't be able to compare the predictions to the current value? 
"""

print(len(df[df['life_expectancy'].isna()]['country'].unique()))
country_drop = df[df['life_expectancy'].isna()]['country'].unique()
country_drop

In [ ]:
rows_to_drop = df[df.country.isin(country_drop)].index
df.drop(index=rows_to_drop,inplace=True)

In [ ]:
df = df.reset_index()
columns_to_drop.append('index')

In [ ]:
df.head()

## Final cleaning

In [ ]:
# Droping useless columns before calculating data
df.drop(columns=columns_to_drop,inplace=True)

In [ ]:
df.head()

In [ ]:
# Replacing missing values by the mean of other values in the column for the given country

for column in df.columns:
    for i in range(df.shape[0]):
        country=df['country'][i]
        if (df[column].isnull()[i]==True):
            df[column][i]=df[df['country']==country][column].mean()

In [ ]:
# Replacing all missing values by the mean of countries in the same status (Developed or Developing)
df_developped=df[(df['status']=='Developed')].fillna(df[(df['status']=='Developed')].mean())
df_developing=df[(df['status']=='Developing')].fillna(df[(df['status']=='Developing')].mean())
df_least_developed=df[(df['status']=='Least_developed')].fillna(df[(df['status']=='Least_developed')].mean())
df1=df_developing.append(df_developped)
df=df1.append(df_least_developed)

In [ ]:
df.isna().sum()

In [ ]:
df.drop(columns=['country','year'],inplace=True)
df.shape

In [ ]:
# Get dummies for status column

df = pd.get_dummies(data=df,columns=['status'], drop_first=True)
df.head()

In [ ]:
df = df.reset_index()
df.drop(columns='index',inplace=True)

In [ ]:
df.shape

## Saving cleaned dataframe

In [ ]:
df.to_csv('../data/life_expectancy_data_cleaned.csv',index=False)